In [1]:
# ['nigga', 'hate', 'love','ass','hell','better']
# #accuracy over all cross-validation folds: [0.6317343173431734, 0.618450184501845, 0.6140221402214022, 0.622140221402214, 0.6137370753323486]
# mean=0.62 std=0.01

# ['nigga', 'hate', 'love','ass','hell','better','bitch','fuck','dick']
# accuracy over all cross-validation folds: [0.6287822878228783, 0.6199261992619927, 0.6154981549815498, 0.6236162361623616, 0.6159527326440177]
# mean=0.62 std=0.00

# ['nigga', 'hate', 'love','ass','hell','better','bitch','fuck','dick','hey','shit','sexy','awesome']
# accuracy over all cross-validation folds: [0.6501845018450184, 0.6789667896678967, 0.6516605166051661, 0.6494464944649446, 0.6676514032496307]
# mean=0.66 std=0.01

from collections import Counter
import numpy as np
import pandas as pd
import re
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer

In [102]:
def load_data(datafile):
    """
    Read your data into a single pandas dataframe where
    - each row is an instance to be classified
    (this could be a tweet, user, or news article, depending on your project)
    - there is a column called `label` which stores the class label (e.g., the true
      category for this row)
    """
    df = pd.read_csv("\\Users\\11977\\elevate-osna-harassment\\osna\\data.csv")[['text', 'hostile']]
    df.columns = ['text', 'label']
    df['label'] = ['hostile' if i==1 else 'nonhostile' for i in df.label]
   # df['directed'] = ['directed' if i==1 else 'nondirected' for i in df.directed]
    return df


In [104]:
df = load_data('~/Dropbox/elevate/harassment/training_data/data.csv.gz')


In [79]:
df.label.value_counts()

hostile       3588
nonhostile    3186
Name: label, dtype: int64

In [144]:
def make_features(df):
    vec = DictVectorizer()
    feature_dicts = []
    words_to_track = ['nigga', 'hate', 'love','ass','hell','better','bitch','fuck','dick','hey','shit','sexy','awesome']
    # will get different model for different features.
    #words_to_track = ['you']
    for i,row in df.iterrows():
        features = {}
        token_counts = Counter(re.sub('\W+', ' ', row['text'].lower()).split())
        for w in words_to_track:
            features[w] = token_counts[w]
        feature_dicts.append(features)
    X = vec.fit_transform(feature_dicts)
    return X, vec
                
X, vec = make_features(df)

In [145]:
X

<6774x13 sparse matrix of type '<class 'numpy.float64'>'
	with 5840 stored elements in Compressed Sparse Row format>

In [146]:
X.shape

(6774, 13)

In [147]:
vec.vocabulary_

{'nigga': 10,
 'hate': 6,
 'love': 9,
 'ass': 0,
 'hell': 7,
 'better': 2,
 'bitch': 3,
 'fuck': 5,
 'dick': 4,
 'hey': 8,
 'shit': 12,
 'sexy': 11,
 'awesome': 1}

In [148]:
X[0,5]

0.0

In [149]:
X[:,3].sum()

410.0

In [150]:
for word, idx in vec.vocabulary_.items():
    print('%20s\t%d' % (word, X[:,idx].sum()))

               nigga	2340
                hate	44
                love	129
                 ass	674
                hell	67
              better	101
               bitch	410
                fuck	923
                dick	153
                 hey	28
                shit	1184
                sexy	12
             awesome	11


In [151]:
vec.get_feature_names()

['ass',
 'awesome',
 'better',
 'bitch',
 'dick',
 'fuck',
 'hate',
 'hell',
 'hey',
 'love',
 'nigga',
 'sexy',
 'shit']

In [152]:
y = np.array(df.label)
Counter(y)

Counter({'nonhostile': 3186, 'hostile': 3588})

In [153]:
y[[0,5,12]]

array(['nonhostile', 'hostile', 'hostile'], dtype=object)

In [154]:
class_names = set(df.label)
for word, idx in vec.vocabulary_.items():
    for class_name in class_names:
        class_idx = np.where(y==class_name)[0]
        print('%20s\t%20s\t%d' % (word, class_name, X[class_idx, idx].sum()))

               nigga	             hostile	922
               nigga	          nonhostile	1418
                hate	             hostile	20
                hate	          nonhostile	24
                love	             hostile	44
                love	          nonhostile	85
                 ass	             hostile	467
                 ass	          nonhostile	207
                hell	             hostile	38
                hell	          nonhostile	29
              better	             hostile	47
              better	          nonhostile	54
               bitch	             hostile	257
               bitch	          nonhostile	153
                fuck	             hostile	596
                fuck	          nonhostile	327
                dick	             hostile	106
                dick	          nonhostile	47
                 hey	             hostile	15
                 hey	          nonhostile	13
                shit	             hostile	480
                shit	          nonhostile	70

In [155]:
clf = LogisticRegression(solver='lbfgs', multi_class='auto')
clf.fit(X, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='auto',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [156]:
clf.coef_

array([[-0.60775075,  0.40264097,  0.20773768, -0.21272082, -0.50157786,
        -0.30095793,  0.15584426, -0.28716009,  0.03647382,  0.8068743 ,
         0.80506142, -1.05330857,  0.65753686]])

In [157]:
coef = [-clf.coef_[0], clf.coef_[0]]
print(coef)

[array([ 0.60775075, -0.40264097, -0.20773768,  0.21272082,  0.50157786,
        0.30095793, -0.15584426,  0.28716009, -0.03647382, -0.8068743 ,
       -0.80506142,  1.05330857, -0.65753686]), array([-0.60775075,  0.40264097,  0.20773768, -0.21272082, -0.50157786,
       -0.30095793,  0.15584426, -0.28716009,  0.03647382,  0.8068743 ,
        0.80506142, -1.05330857,  0.65753686])]


In [158]:
clf.classes_

array(['hostile', 'nonhostile'], dtype=object)

In [159]:
for ci, class_name in enumerate(clf.classes_):
    print('coefficients for %s' % class_name)
    display(pd.DataFrame([coef[ci]], columns=vec.get_feature_names()))

coefficients for hostile


,ass,awesome,better,bitch,dick,fuck,hate,hell,hey,love,nigga,sexy,shit
0,0.607751,-0.402641,-0.207738,0.212721,0.501578,0.300958,-0.155844,0.28716,-0.036474,-0.806874,-0.805061,1.053309,-0.657537


coefficients for nonhostile


,ass,awesome,better,bitch,dick,fuck,hate,hell,hey,love,nigga,sexy,shit
0,-0.607751,0.402641,0.207738,-0.212721,-0.501578,-0.300958,0.155844,-0.28716,0.036474,0.806874,0.805061,-1.053309,0.657537


In [160]:
features = vec.get_feature_names()
for ci, class_name in enumerate(clf.classes_):
    print('top features for class %s' % class_name)
    for fi in coef[ci].argsort()[::-1]: # descending order.
        print('%20s\t%.2f' % (features[fi], coef[ci][fi]))

top features for class hostile
                sexy	1.05
                 ass	0.61
                dick	0.50
                fuck	0.30
                hell	0.29
               bitch	0.21
                 hey	-0.04
                hate	-0.16
              better	-0.21
             awesome	-0.40
                shit	-0.66
               nigga	-0.81
                love	-0.81
top features for class nonhostile
                love	0.81
               nigga	0.81
                shit	0.66
             awesome	0.40
              better	0.21
                hate	0.16
                 hey	0.04
               bitch	-0.21
                hell	-0.29
                fuck	-0.30
                dick	-0.50
                 ass	-0.61
                sexy	-1.05


In [161]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score

kf = KFold(n_splits=5, shuffle=True, random_state=42)
accuracies = []
for train, test in kf.split(X):
    clf.fit(X[train], y[train])
    pred = clf.predict(X[test])
    accuracies.append(accuracy_score(y[test], pred))
    
    
print('accuracy over all cross-validation folds: %s' % str(accuracies))
print('mean=%.2f std=%.2f' % (np.mean(accuracies), np.std(accuracies)))

accuracy over all cross-validation folds: [0.6501845018450184, 0.6789667896678967, 0.6516605166051661, 0.6494464944649446, 0.6676514032496307]
mean=0.66 std=0.01
